In [1]:
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
def Character_Dataset():
  df = pd.read_csv('emnist-letters.csv')
  for feature in df.columns:# normalizing feature value other the output
    if feature != 'label':
      df[feature] = (df[feature])/255
  m=df.shape[0]
  n=len(df.columns)
  traincount = 5000
  testend = 6000
  X_train = df.iloc[:traincount, 1:n].to_numpy(dtype=np.float64)
  Y_train = pd.get_dummies(df['label']).iloc[:traincount,:].to_numpy(dtype=np.float64)
  X_test = df.iloc[traincount:testend, 1:n].to_numpy(dtype=np.float64)
  Y_test = pd.get_dummies(df['label']).iloc[traincount:testend,:].to_numpy(dtype=np.float64)
  X_train = X_train.reshape((5000,28,28,1))
  X_test = X_test.reshape((1000,28,28,1))
  Y_train = Y_train.reshape((5000,26))
  Y_test = Y_test.reshape((1000,26))
  return (X_train,Y_train), (X_test,Y_test)

In [5]:
M = Sequential()
M.add(Conv2D(20,kernel_size=(3,3),activation="relu",input_shape=(28,28,1)))
M.add(MaxPool2D(pool_size=(2,2),strides=(1,1)))
M.add(Conv2D(40,kernel_size=(3,3),activation="relu"))
M.add(MaxPool2D(pool_size=(2,2),strides=(1,1)))
M.add(Flatten())
M.add(Dense(100,activation="relu"))
M.add(Dense(26,activation="softmax"))
print(M.summary())
(X_train, Y_train), (X_test,Y_test) = Character_Dataset()
M.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
M.fit(X_train,Y_train,validation_data=(X_test, Y_test),epochs=10)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 20)        200       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 25, 25, 20)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 23, 23, 40)        7240      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 22, 22, 40)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 19360)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               1936100   
_________________________________________________________________
dense_5 (Dense)              (None, 26)               

In [6]:
print("No of layers : "+str(len(M.layers)))
for i in M.layers:
  print(i.get_weights())

No of layers : 7
[array([[[[-0.18589565, -0.10802475,  0.20368928, -0.28563148,
           0.0803261 , -0.10360219,  0.07215033, -0.04782815,
           0.24682404,  0.10796101,  0.07943764,  0.0338226 ,
          -0.16305515, -0.12662224, -0.20652667, -0.25193164,
           0.09259631, -0.0093121 ,  0.19945464, -0.06093852]],

        [[-0.06696849,  0.0391739 ,  0.20260487, -0.15260558,
          -0.1869637 ,  0.07033505,  0.19063413,  0.02502155,
          -0.12287801,  0.09202819, -0.13084587, -0.05593337,
          -0.13107845, -0.09307594, -0.10943262, -0.02400215,
           0.2094621 ,  0.05039958,  0.21267852, -0.05298243]],

        [[-0.17328154,  0.20553875,  0.21372052, -0.02029886,
          -0.19286594,  0.14009589,  0.02800955, -0.00212646,
          -0.13743389, -0.2147488 , -0.12631644, -0.25204998,
          -0.13639235,  0.18794821, -0.12022749,  0.26839778,
           0.04949556, -0.07052053,  0.12617153, -0.0106536 ]]],


       [[[ 0.21976162, -0.21239358,  0.08

[array([[-0.00384111,  0.01463793,  0.03429354, ...,  0.00766641,
        -0.01984668,  0.01519286],
       [-0.00520681, -0.01569425,  0.01672608, ...,  0.00159401,
        -0.00469867, -0.03261628],
       [-0.02954931,  0.01277431, -0.03330268, ..., -0.00213126,
        -0.03681884,  0.023892  ],
       ...,
       [ 0.04029781,  0.0018552 ,  0.04905686, ...,  0.01158992,
         0.00179692, -0.04064826],
       [ 0.07017702,  0.00881445,  0.00158337, ..., -0.003954  ,
         0.00060618, -0.03409978],
       [ 0.0154651 , -0.00087771,  0.01442313, ...,  0.00065886,
        -0.02043435, -0.0119796 ]], dtype=float32), array([-1.85072199e-02, -6.00491464e-03,  1.08738706e-04,  9.26331151e-03,
        8.68976349e-05, -6.09122356e-03, -6.00424642e-03, -5.95859019e-03,
       -1.47496862e-02, -6.00458588e-03, -6.00474654e-03, -8.25239904e-03,
       -1.16044162e-02, -1.86920278e-02, -6.00420916e-03, -6.00379938e-03,
        1.67105328e-02,  6.55944552e-03, -5.69026219e-03,  1.00086322e

In [7]:
Y = M.predict(X_test)
print(np.argmax(Y[0:10],axis=1))
print(np.argmax(Y_test[0:10],axis=1))
print("Total correctly predicted : "+str(np.sum((np.argmax(Y,axis=1)==np.argmax(Y_test,axis=1))))+" out of "+str(Y_test.shape[0]))

[10 15 24  5 14 25 10 15 18 24]
[10 15 19  5 14 25 10 15 18 24]
Total correctly predicted : 859 out of 1000
